In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import numpy as np
import csv
import pandas as pd

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--disable-notifications")
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
f = open(r'C:\Users\Admin\Desktop\datasets\facebook_groups_dataset.csv', 'a', encoding="UTF8", newline='')
writer = csv.writer(f)
header = ['Post','Reacts_Count','Most_Reacts','Image','Group']
writer.writerow(header)
driver.get('https://www.facebook.com')
time.sleep(3)
email = driver.find_element(By.ID, 'email')
password = driver.find_element(By.ID, 'pass')


# put your login info here 
email.send_keys('your email here')
password.send_keys('your password')


login = driver.find_element(By.CLASS_NAME, '_42ft._4jy0._6lth._4jy6._4jy1.selected._51sy')
login.click()
time.sleep(2)
groups = [
        'https://www.facebook.com/groups/721480409133053/?ref=share&mibextid=S66gvF',
        'https://www.facebook.com/groups/227495968893129/?ref=share&mibextid=S66gvF',
        'https://www.facebook.com/groups/melle5er/?ref=share&mibextid=S66gvF',
        'https://www.facebook.com/groups/1315249288876903/?ref=share&mibextid=S66gvF',
        'https://www.facebook.com/groups/4774746349290676/',
        'https://www.facebook.com/groups/ibilieve',
        'https://www.facebook.com/groups/mekech.tounsi.ken.mee',
         ]
for group in groups:
    
    driver.get(group)
    time.sleep(4)
    scrolled = 0
    posts = set()  # Use a set to store unique posts

    # There is a limit for scrolling depending on your RAM, in my case (4gb ram) google chrome crashes after about 90 
    while scrolled < 80:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
        scrolled += 1
        time.sleep(4)

        new_posts = driver.find_elements(By.CSS_SELECTOR, '.x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
        for post in new_posts:
            # Check if the post is already captured
            if post in posts:
                continue

            data = []
            # Find the actual post element within the post element
            try:
                actual_post = post.find_element(By.CSS_SELECTOR, ".xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x1vvkbs").text
            except NoSuchElementException:
                continue
            if actual_post == '':
                try:
                    actual_post = post.find_element(By.CSS_SELECTOR, '.x5yr21d.xyqdw3p').text
                except NoSuchElementException:
                    continue
            # Find the number of reacts element within the post element
            reacts_element = post.find_element(By.CSS_SELECTOR, '.xrbpyxo.x6ikm8r.x10wlt62.xlyipyv.x1exxlbk')
            number_of_reacts = reacts_element.text


            # Find the most used react
            reacts_element = post.find_element(By.CSS_SELECTOR, ".x6s0dn4.x78zum5.x1e558r4")
            reacts = reacts_element.find_elements(By.TAG_NAME, 'span')[0].find_elements(By.TAG_NAME, 'span')[0].find_elements(By.TAG_NAME, 'span')[0].find_elements(By.TAG_NAME, 'div')[0]
            react = reacts.get_attribute('aria-label').split(':')[0]

            # Scrape picture URL if found
            try:
                image_element = post.find_element(By.CSS_SELECTOR, '.x1i10hfl.x1qjc9v5.xjbqb8w.xjqpnuy.xa49m3k.xqeqjp1.x2hbi6w.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xdl72j9.x2lah0s.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xeuugli.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1o1ewxj.x3x9cwd.x1e5q0jg.x13rtm0m.x1q0g3np.x87ps6o.x1lku1pv.x1a2a7pz.x1lliihq.x1pdlv7q')
                image = image_element.get_attribute("href")
            except NoSuchElementException:
                image = 'None'

            data.append(actual_post)
            data.append(number_of_reacts)
            data.append(react)
            data.append(image)
            data.append(group)
            writer.writerow(data)

            # Add the post to the set to avoid duplicates
            posts.add(post)
            if len(posts)%100 == 0:
                print(f'scraped {len(posts)} posts.')
        if scrolled%10 == 0:
            print(f'scrolled {scrolled} times.')

driver.quit()
f.close()